In [41]:
import pandas as pd
import folium
file = ('case1.xlsx')
df1 = pd.read_excel(file,skiprows = 1, header = 0)
df1['Latitude'] = 55.539306
df1['Longitude'] = 51.856451
df1['point'] = 'A'
df1 = df1[df1.index > 1]
file = ('case2.xlsx')
df2 = pd.read_excel(file,skiprows = 1, header = 0)
df2['Latitude'] = 55.654578
df2['Longitude'] = 51.800072
df2['point'] = 'B'
df2 = df2[df2.index > 1]
file = ('case3.xlsx')
df3 = pd.read_excel(file,skiprows = 1, header = 0)
df3['Latitude'] = 55.613193
df3['Longitude'] = 51.784821
df3['point'] = 'C'
df3 = df3[df3.index > 1]
file = ('case4.xlsx')
df4 = pd.read_excel(file,skiprows = 1, header = 0)
df4['Latitude'] = 55.598983
df4['Longitude'] = 51.771936
df4['point'] = 'D'
df4 = df4[df4.index > 1]
file = ('case5.xlsx')
df5 = pd.read_excel(file,skiprows = 1, header = 0)
df5['Latitude'] = 55.650091
df5['Longitude'] = 51.852687
df5['point'] = 'E'
df5 = df5[df5.index > 1]
file = ('case6.xlsx')
df6 = pd.read_excel(file,skiprows = 1, header = 0)
df6['Latitude'] = 55.622944
df6['Longitude'] = 51.82557
df6['point'] = 'F'
df6 = df6[df6.index > 1]
result = pd.concat([df1, df2,df3, df4,df5, df6], ignore_index=True)
result.head()

,Unnamed: 0,Показатель:,"CO, мг/м³","NO, мг/м³","NO2, мг/м³","NH3, мг/м³","SO2, мг/м³","H2S, мг/м³","T внеш., °C","V ветра, м/с",...,"Нонан, мг/м³","Декан, мг/м³","О,м,п-ксилолы, мг/м³","М-ксилол, мг/м³","П-ксилол, мг/м³","Формальдегид, мг/м³","CO2, мг/м³","Нафталин, мг/м³","Бутилацетат, мг/м³","Бутанол, мг/м³"
0,01.06.2023 00:00,NaN,"0,2","0,001","0,001","0,014","0,003","0,0004","16,8","<0,5 (0,2)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01.06.2023 00:30,NaN,"0,2","0,002","0,001","0,014","0,003","0,0004","15,8","<0,5 (0,3)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01.06.2023 01:00,NaN,"0,2","0,002","0,002","0,014","0,002","0,0006","15,3","<0,5 (0,1)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01.06.2023 01:30,NaN,"0,2","0,002","0,001","0,014","0,003","0,0005","14,8","<0,5 (0,0)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01.06.2023 02:00,NaN,"0,2","0,002","0,001","0,015","0,003","0,0005","15,2","<0,5 (0,4)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
result.drop('Показатель:', axis=1, inplace=True)
result = result.rename({'Unnamed: 0':'date_time'}, axis=1)
new_columns = [col.split(',')[0] for col in result.columns]
result.columns = new_columns

In [43]:
desired_columns = ['point', 'Latitude', 'Longitude', 'date_time']

other_columns = [col for col in result.columns if col not in desired_columns]
new_order = desired_columns + other_columns

result = result[new_order]

In [44]:
exclude_columns = ['point', 'Latitude', 'Longitude', 'date_time', 'V ветра', 'D ветра']

numeric_columns = [col for col in result.columns if col not in exclude_columns]
result[numeric_columns] = result[numeric_columns].apply(lambda x: x.str.replace(',', '.'))
result[numeric_columns] = result[numeric_columns].apply(pd.to_numeric)

In [45]:
mac_df = pd.read_excel('/Users/a.germanskiy/Downloads/substances macs.xlsx')
filtered_rows = []
for indicator, mac in mac_df.itertuples(index=False):
    indicator_column = result[indicator]  # Выбираем столбец по текущему Indicator
    mask = indicator_column > mac
    if mask.any():
        real_mac = mac / 0.9
        filtered_data = result.loc[mask, :].copy()
        filtered_data['ExceededIndicator'] = indicator  # Добавляем новый столбец
        filtered_data['Indicator_mac'] = real_mac  # Добавляем новый столбец
        filtered_data['excess_value'] = round(filtered_data[indicator] / real_mac, 2)
        filtered_rows.append(filtered_data)

# Объединяем отфильтрованные строки в один датафрейм
filtered_df = pd.concat(filtered_rows)

In [46]:
filtered_df

,point,Latitude,Longitude,date_time,CO,NO,NO2,NH3,SO2,H2S,...,М-ксилол,П-ксилол,Формальдегид,CO2,Нафталин,Бутилацетат,Бутанол,ExceededIndicator,Indicator_mac,excess_value
2726,B,55.654578,51.800072,28.06.2023 19:00,0.200,0.005,0.004,0.006,0.0158,0.0084,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H2S,0.008889,0.94
2729,B,55.654578,51.800072,28.06.2023 20:30,0.000,0.000,0.020,0.000,0.0116,0.0081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H2S,0.008889,0.91
2732,B,55.654578,51.800072,28.06.2023 22:00,0.000,0.000,0.012,0.000,0.0131,0.0083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H2S,0.008889,0.93
2733,B,55.654578,51.800072,28.06.2023 22:30,0.000,0.000,0.026,0.000,0.0132,0.0084,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H2S,0.008889,0.94
2734,B,55.654578,51.800072,28.06.2023 23:00,0.000,0.000,0.031,0.000,0.0126,0.0083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H2S,0.008889,0.93
2735,B,55.654578,51.800072,28.06.2023 23:30,0.000,0.000,0.037,0.000,0.0128,0.0092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H2S,0.008889,1.03
2736,B,55.654578,51.800072,29.06.2023 00:00,0.000,0.000,0.028,0.000,0.0119,0.0087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H2S,0.008889,0.98
2737,B,55.654578,51.800072,29.06.2023 00:30,0.000,0.000,0.020,0.000,0.0119,0.0084,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H2S,0.008889,0.94
2740,B,55.654578,51.800072,29.06.2023 02:00,0.000,0.000,0.011,0.000,0.0115,0.0082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H2S,0.008889,0.92
2787,C,55.613193,51.784821,01.06.2023 01:30,0.308,NaN,NaN,NaN,0.0134,0.0090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H2S,0.008889,1.01


In [32]:
import pandas as pd
import folium
file = ('case1.xlsx')
df1 = pd.read_excel(file,skiprows = 1, header = 0)
df1['Latitude'] = 55.539306
df1['Longitude'] = 51.856451
df1['point'] = 'A'
df1 = df1[df1.index > 1]
file = ('case2.xlsx')
df2 = pd.read_excel(file,skiprows = 1, header = 0)
df2['Latitude'] = 55.654578
df2['Longitude'] = 51.800072
df2['point'] = 'B'
df2 = df2[df2.index > 1]
file = ('case3.xlsx')
df3 = pd.read_excel(file,skiprows = 1, header = 0)
df3['Latitude'] = 55.613193
df3['Longitude'] = 51.784821
df3['point'] = 'C'
df3 = df3[df3.index > 1]
file = ('case4.xlsx')
df4 = pd.read_excel(file,skiprows = 1, header = 0)
df4['Latitude'] = 55.598983
df4['Longitude'] = 51.771936
df4['point'] = 'D'
df4 = df4[df4.index > 1]
file = ('case5.xlsx')
df5 = pd.read_excel(file,skiprows = 1, header = 0)
df5['Latitude'] = 55.650091
df5['Longitude'] = 51.852687
df5['point'] = 'E'
df5 = df5[df5.index > 1]
file = ('case6.xlsx')
df6 = pd.read_excel(file,skiprows = 1, header = 0)
df6['Latitude'] = 55.622944
df6['Longitude'] = 51.82557
df6['point'] = 'F'
df6 = df6[df6.index > 1]
result = pd.concat([df1, df2,df3, df4,df5, df6], ignore_index=True)
result.head()

,Unnamed: 0,Показатель:,"CO, мг/м³","NO, мг/м³","NO2, мг/м³","NH3, мг/м³","SO2, мг/м³","H2S, мг/м³","T внеш., °C","V ветра, м/с",...,"Нонан, мг/м³","Декан, мг/м³","О,м,п-ксилолы, мг/м³","М-ксилол, мг/м³","П-ксилол, мг/м³","Формальдегид, мг/м³","CO2, мг/м³","Нафталин, мг/м³","Бутилацетат, мг/м³","Бутанол, мг/м³"
0,01.06.2023 00:00,NaN,"0,2","0,001","0,001","0,014","0,003","0,0004","16,8","<0,5 (0,2)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01.06.2023 00:30,NaN,"0,2","0,002","0,001","0,014","0,003","0,0004","15,8","<0,5 (0,3)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01.06.2023 01:00,NaN,"0,2","0,002","0,002","0,014","0,002","0,0006","15,3","<0,5 (0,1)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01.06.2023 01:30,NaN,"0,2","0,002","0,001","0,014","0,003","0,0005","14,8","<0,5 (0,0)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01.06.2023 02:00,NaN,"0,2","0,002","0,001","0,015","0,003","0,0005","15,2","<0,5 (0,4)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
result['Unnamed: 0'] = pd.to_datetime(result['Unnamed: 0'])

In [34]:
initial_latitude = 55.539306
initial_longitude = 51.856451

# Создание карты
m = folium.Map(location=[initial_latitude, initial_longitude], zoom_start=10)

# Добавление маркеров на карту
l = []
for index, row in result.iterrows():
    if row['Latitude'] in l and row['Longitude'] in l:
        pass
    else:
        folium.Marker([row['Latitude'], row['Longitude']], popup=row['point']).add_to(m)
        l.append(row['Latitude'])
        l.append(row['Longitude'])
folium.Marker([55.605673, 51.962679]).add_to(m)

# Сохранение карты в HTML-файл
m.save('map.html')

In [35]:
result['Unnamed: 0'] = pd.to_datetime(result['Unnamed: 0'])
result['D ветра, °'] = result['D ветра, °'].str.replace(r'\D', '', regex=True)
result['D ветра, °']= pd.to_numeric(result['D ветра, °'], errors='coerce')
# result['V ветра, м/с']= pd.to_numeric(result['V ветра, м/с'], errors='coerce')
result['Latitude'] = pd.to_numeric(result['Latitude'], errors='coerce')
result['Longitude'] = pd.to_numeric(result['Longitude'], errors='coerce')
result['V ветра, м/с'] = result['V ветра, м/с'].str.extract(r'\((.*?)\)')[0].fillna(result['V ветра, м/с'].str.extract(r'\((.*?)\)|([^()]+)')[1])
result['V ветра, м/с'] = result['V ветра, м/с'].str.replace(',', '.')
result['V ветра, м/с'] = pd.to_numeric(result['V ветра, м/с'], errors='coerce')

In [36]:
result['CO, мг/м³'] = result['CO, мг/м³'].str.replace(',', '.')
result['CO, мг/м³'] = pd.to_numeric(result['CO, мг/м³'], errors='coerce')
result['NO, мг/м³'] = result['NO, мг/м³'].str.replace(',', '.')
result['NO, мг/м³'] = pd.to_numeric(result['NO, мг/м³'], errors='coerce')
result['NO2, мг/м³'] = result['NO2, мг/м³'].str.replace(',', '.')
result['NO2, мг/м³'] = pd.to_numeric(result['NO2, мг/м³'], errors='coerce')
result['NH3, мг/м³'] = result['NH3, мг/м³'].str.replace(',', '.')
result['NH3, мг/м³'] = pd.to_numeric(result['NH3, мг/м³'], errors='coerce')
result['SO2, мг/м³'] = result['SO2, мг/м³'].str.replace(',', '.')
result['SO2, мг/м³'] = pd.to_numeric(result['SO2, мг/м³'], errors='coerce')
result['H2S, мг/м³'] = result['H2S, мг/м³'].str.replace(',', '.')
result['H2S, мг/м³'] = pd.to_numeric(result['H2S, мг/м³'], errors='coerce')

In [37]:
# hdf1 = pd.DataFrame()
# for index, row in result.iterrows():
#     if row['D ветра, °'] < 22.5:
#         row['direction'] = 'N'
#     elif row['D ветра, °'] < 67.5:
#         row['direction'] = 'NE'
#     elif row['D ветра, °'] < 112.5:
#         row['direction'] = 'E'
#     elif row['D ветра, °'] < 157.5:
#         row['direction'] = 'SE'
#     elif row['D ветра, °'] < 202.5:
#         row['direction'] = 'S'
#     elif row['D ветра, °'] < 247.5:
#         row['direction'] = 'SW'
#     elif row['D ветра, °'] < 292.5:
#         row['direction'] = 'W'
#     elif row['D ветра, °'] < 337.5:
#         row['direction'] = 'NW'
#     else:
#         row['direction'] = 'N'
#     hdf1 = pd.concat([hdf1, row.to_frame().T])
# hdf1


In [38]:
# hdf = pd.DataFrame()
# for index, row in hdf1.iterrows():
#     if row['CO, мг/м³'] >= hdf1['CO, мг/м³'].quantile(0.9):
#         hdf = pd.concat([hdf,row.to_frame().T])
# a = hdf.groupby(['point','direction']).count().reset_index()
# a = a[['point', 'direction', 'CO, мг/м³']]
# for index, row in hdf1.iterrows():
#     if row['NO, мг/м³'] >= hdf1['NO, мг/м³'].quantile(0.9):
#         hdf = pd.concat([hdf,row.to_frame().T])
# # a = pd.concat([a,hdf.groupby(['point','direction']).count().reset_index()[['point', 'direction', 'NO, мг/м³']]])
# a = a.merge(hdf, on=['point','direction'], how='inner')
# a
# hdf.groupby(['point','direction']).count()
# for index, row in hdf1.iterrows():
#     if row['NO2, мг/м³'] >= hdf1['NO2, мг/м³'].quantile(0.9):
#         hdf = pd.concat([hdf,row.to_frame().T])
# hdf.groupby(['point','direction']).count()
# for index, row in hdf1.iterrows():
#     if row['NH3, мг/м³'] >= hdf1['NH3, мг/м³'].quantile(0.9):
#         hdf = pd.concat([hdf,row.to_frame().T])
# hdf.groupby(['point','direction']).count()
# for index, row in hdf1.iterrows():
#     if row['SO2, мг/м³'] >= hdf1['SO2, мг/м³'].quantile(0.9):
#         hdf = pd.concat([hdf,row.to_frame().T])
# for index, row in hdf1.iterrows():
#     if row['H2S, мг/м³'] >= hdf1['H2S, мг/м³'].quantile(0.9):
#         hdf = pd.concat([hdf,row.to_frame().T])
# hdf.groupby(['point','direction']).count()

In [39]:
given_period = result[result['Unnamed: 0'] == '2023-06-28 22:30:00']
given_period = given_period.fillna(0)

In [40]:
import folium
import math
from folium.vector_layers import PolyLine
from folium.plugins import TimeSliderChoropleth
initial_coords = (55.539306, 51.856451)
color2 = 'blue'
m = folium.Map(location=initial_coords, zoom_start=11)

for index, row in given_period.iterrows():
    for i in range(3):
        for j in range(3):
            if row['D ветра, °'] > 0:
                start_point = (row['Latitude'] + i/100 - j/100, row['Longitude'] - i/100 - j/100)
                angle_degrees = row['D ветра, °']
#                 angle_degrees = 45
                angle_degrees1 = angle_degrees + 30 + 180
                angle_degrees2 = angle_degrees + 180 - 30
                angle_radians = math.radians(angle_degrees)
                angle_radians1 = math.radians(angle_degrees1)
                angle_radians2 = math.radians(angle_degrees2)
                vector_length = max(0.003,row['V ветра, м/с'] * 0.008)
                end_lat = start_point[0] + vector_length * math.cos(angle_radians)
                end_lon = start_point[1] + vector_length * math.sin(angle_radians)
                end_point = (end_lat, end_lon)
                end_1at_1 = end_point[0] + vector_length * 0.5 * math.cos(angle_radians1)
                end_1on_1 = end_point[1] + vector_length * 0.5 * math.sin(angle_radians1)
                end_1at_2 = end_point[0] + vector_length * 0.5 * math.cos(angle_radians2)
                end_1on_2 = end_point[1] + vector_length * 0.5 * math.sin(angle_radians2)
                end_1 = (end_1at_1,end_1on_1)
                end_2 = (end_1at_2,end_1on_2)
                if row['V ветра, м/с'] <= 1.2:
                    color1 = 'blue'
                elif row['V ветра, м/с'] <= 1.9:
                    color1 = 'orange'
                elif row['V ветра, м/с'] <= 3.0:
                    color1 = 'red'
                elif row['V ветра, м/с'] <= 3.5:
                    color = 'dark red'
                else:
                    color1 = 'black'
                vector_line = PolyLine(locations=[start_point, end_point], color=color1, weight=2)
                vector_line1 = PolyLine(locations=[end_point, end_1], color=color1, weight=2)
                vector_line2 = PolyLine(locations=[end_point, end_2], color=color1, weight=2)
                if row['CO, мг/м³'] >= result['CO, мг/м³'].quantile(0.9):
                    color1 = 'red'
                    tag = 'CO'
                elif row['NO, мг/м³'] >= result['NO, мг/м³'].quantile(0.9):
                    color2 = 'red'
                    tag = 'NO'
                elif row['NO2, мг/м³'] >= result['NO2, мг/м³'].quantile(0.9):
                    color2 = 'red'
                    tag = 'NO2'
                elif row['NH3, мг/м³'] >= result['NH3, мг/м³'].quantile(0.9):
                    color1 = 'red'
                    tag = 'NH3'
                elif row['SO2, мг/м³'] >= result['SO2, мг/м³'].quantile(0.9):
                    color2 = 'red'
                    tag = 'SO2'
                elif row['H2S, мг/м³'] >= result['H2S, мг/м³'].quantile(0.9):
                    color2 = 'red'
                    tag = 'H2S'
                if color2 == 'red':
                    circle_center = (row['Latitude'], row['Longitude'])
                    radius_meters = 400  # Радиус 1 км
                    circle = folium.Circle(
                        location=circle_center,
                        radius=radius_meters,
                        color=color2,  # Цвет обводки круга
                        fill=True,     # Заполнить круг цветом
                        fill_color=color2,  # Цвет заполнения круга
                        fill_opacity=0.07,
                        stroke = False # Прозрачность заполнения
                    )
                    circle.add_to(m)
                vector_line.add_to(m)
                vector_line1.add_to(m)
                vector_line2.add_to(m)
m

In [413]:
# print(result['Unnamed: 0'].count())
# print(result[result['V ветра, м/с'] > result['V ветра, м/с'].quantile(0.5)]['Unnamed: 0'].count(), result['V ветра, м/с'].quantile(0.5))
# print(result[result['V ветра, м/с'] > result['V ветра, м/с'].quantile(0.7)]['Unnamed: 0'].count(), result['V ветра, м/с'].quantile(0.7))
# print(result[result['V ветра, м/с'] > result['V ветра, м/с'].quantile(0.9)]['Unnamed: 0'].count(), result['V ветра, м/с'].quantile(0.9))
# print(result[result['V ветра, м/с'] > result['V ветра, м/с'].quantile(0.95)]['Unnamed: 0'].count(), result['V ветра, м/с'].quantile(0.95))

8352
3890 1.2
2346 1.9
772 3.0
396 3.5


In [239]:
# import math
# from geopy.distance import geodesic

# # Coordinates of the first point (latitude, longitude)
# point1 = (55.539306, 51.856451)  # London

# # Coordinates of the second point (latitude, longitude)
# point2 = (55.654578, 51.800072)  # New York

# # Calculate the bearing between the points
# lat1, lon1 = math.radians(point1[0]), math.radians(point1[1])
# lat2, lon2 = math.radians(point2[0]), math.radians(point2[1])

# delta_lon = lon2 - lon1

# y = math.sin(delta_lon) * math.cos(lat2)
# x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon)

# bearing = math.atan2(y, x)
# bearing_degrees = (math.degrees(bearing) + 360) % 360

# print("Bearing from the first point to the second:", bearing_degrees, "degrees")

Bearing from the first point to the second: 344.5754222558657 degrees


ValueError: could not convert string to float: '01.06.2023 00:00'

In [225]:
result.head()

,Unnamed: 0,Показатель:,"CO, мг/м³","NO, мг/м³","NO2, мг/м³","NH3, мг/м³","SO2, мг/м³","H2S, мг/м³","T внеш., °C","V ветра, м/с","D ветра, °","H внеш., %","P атм., мм.рт.ст.","R осадк., мм/ч",Latitude,Longitude,point,"NOx, мг/м³","Фенол, мг/м³","Бензол, мг/м³","Толуол, мг/м³","Этилбензол, мг/м³","М,п-ксилол, мг/м³","О-ксилол, мг/м³","Стирол, мг/м³","Хлорбензол, мг/м³","Этилена оксид, мг/м³","CH4, мг/м³","Этан, мг/м³","Этилен, мг/м³","Пропен, мг/м³","Пропан, мг/м³","Изобутан, мг/м³","Бутан, мг/м³","Изобутилен, мг/м³","Бутен-1, мг/м³","Бутен-2, мг/м³","Пентан, мг/м³","Изопентан, мг/м³","Гексан, мг/м³","Гептан, мг/м³","Октан, мг/м³","Нонан, мг/м³","Декан, мг/м³","О,м,п-ксилолы, мг/м³","М-ксилол, мг/м³","П-ксилол, мг/м³","Формальдегид, мг/м³","CO2, мг/м³","Нафталин, мг/м³","Бутилацетат, мг/м³","Бутанол, мг/м³"
0,01.06.2023 00:00,NaN,"0,2","0,001","0,001","0,014","0,003","0,0004","16,8","<0,5 (0,2)",106 (В),83,"745,1","0,0",55.539306,51.856451,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01.06.2023 00:30,NaN,"0,2","0,002","0,001","0,014","0,003","0,0004","15,8","<0,5 (0,3)",106 (В),85,"745,1","0,0",55.539306,51.856451,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01.06.2023 01:00,NaN,"0,2","0,002","0,002","0,014","0,002","0,0006","15,3","<0,5 (0,1)",107 (В),87,"745,0","0,0",55.539306,51.856451,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01.06.2023 01:30,NaN,"0,2","0,002","0,001","0,014","0,003","0,0005","14,8","<0,5 (0,0)",Ш,88,"745,1","0,0",55.539306,51.856451,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01.06.2023 02:00,NaN,"0,2","0,002","0,001","0,015","0,003","0,0005","15,2","<0,5 (0,4)",134 (Ю-В),90,"745,2","0,0",55.539306,51.856451,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [240]:
# import math
# from geopy.distance import geodesic
# hdf1 = result[['Latitude', 'Longitude', 'point']].drop_duplicates()
# hdf2 = hdf1.merge(hdf1, how='cross')
# hdf2
# def distancee(row):
#     point_a = (row['Latitude_x'], row['Longitude_x'])
#     point_a = (row['Latitude_y'], row['Longitude_y'])
#     distance_km = geodesic(point_a, point_b).kilometers
#     return distance_km
# hdf2['distance'] = hdf2.apply(distancee, axis=1)
# def ugol(row):
#     lat1, lon1 = math.radians(row['Latitude_x']), math.radians(row['Longitude_x'])
#     lat2, lon2 = math.radians(row['Latitude_y']), math.radians(row['Longitude_y'])
#     delta_lon = lon2 - lon1
#     y = math.sin(delta_lon) * math.cos(lat2)
#     x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon)
#     bearing = math.atan2(y, x)
#     bearing_degrees = (math.degrees(bearing) + 360) % 360
#     return bearing_degrees
# hdf2['degrees'] = hdf2.apply(ugol, axis=1)
# hdf2

,Latitude_x,Longitude_x,point_x,Latitude_y,Longitude_y,point_y,distance,degrees
0,55.539306,51.856451,A,55.539306,51.856451,A,13.316799,0.000000
1,55.539306,51.856451,A,55.654578,51.800072,B,0.000000,344.575422
2,55.539306,51.856451,A,55.613193,51.784821,C,4.706659,331.305004
3,55.539306,51.856451,A,55.598983,51.771936,D,6.438423,321.349212
4,55.539306,51.856451,A,55.650091,51.852687,E,3.349666,358.901737
5,55.539306,51.856451,A,55.622944,51.825570,F,3.870737,348.224387
6,55.654578,51.800072,B,55.539306,51.856451,A,13.316799,164.528905
7,55.654578,51.800072,B,55.654578,51.800072,B,0.000000,0.000000
8,55.654578,51.800072,B,55.613193,51.784821,C,4.706659,191.757324
9,55.654578,51.800072,B,55.598983,51.771936,D,6.438423,195.957745


In [285]:
# result.head()
# result
# result['CO, мг/м³'] = result['CO, мг/м³'].str.replace(',', '.')
# result['CO, мг/м³'] = pd.to_numeric(result['CO, мг/м³'], errors='coerce')
# result['CO, мг/м³'].quantile(0.95)

0.7

In [ ]:
# pd.set_option('display.max_columns', None)
# hdf = result[result['CO, мг/м³'] > result['CO, мг/м³'].quantile(0.8)]
# hdf.head()

In [304]:
# result['D ветра, °'] = result['D ветра, °'].str.replace(r'\D', '', regex=True)
# result['D ветра, °']= pd.to_numeric(result['D ветра, °'], errors='coerce')
# result['Unnamed: 0'] = pd.to_datetime(result['Unnamed: 0'])
# result

AttributeError: Can only use .str accessor with string values!

In [250]:
# ind = 0
# dff1 = pd.DataFrame()
# dff2 = pd.DataFrame()
# pd.set_option('display.max_columns', None)
# hdf = result[result['CO, мг/м³'] > result['CO, мг/м³'].quantile(0.8)]
# for index, row in hdf.iterrows():
#     for index2, row2 in hdf.iterrows():
#         if pd.to_datetime(row2['Unnamed: 0']) <= pd.to_datetime(row['Unnamed: 0']) + pd.DateOffset(hours=3) and pd.to_datetime(row2['Unnamed: 0']) >= pd.to_datetime(row['Unnamed: 0']) and row2['point'] != row['point']:
#             for index3, row3 in hdf2.iterrows():
#                 if row3['point_x'] == row['point'] and row3['point_y'] == row2['point'] and (row3['degrees'] 
#                 >= row['D ветра, °'] - 22.5 and row3['degrees'] <= row['D ветра, °'] + 22.5):
#                     row['ind'] = ind
#                     row2['ind'] = ind
#                     dff1 = pd.concat([dff1, row.to_frame().T])
#                     dff2 = pd.concat([dff2, row2.to_frame().T])
#                     ind+=1
#                     print(ind)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225


In [267]:
# result = pd.merge(dff1, dff2, on='ind')
# result = result[(result['point_x'] == 'D')]
# result

,Unnamed: 0_x,Показатель:_x,"CO, мг/м³_x","NO, мг/м³_x","NO2, мг/м³_x","NH3, мг/м³_x","SO2, мг/м³_x","H2S, мг/м³_x","T внеш., °C_x","V ветра, м/с_x","D ветра, °_x","H внеш., %_x","P атм., мм.рт.ст._x","R осадк., мм/ч_x",Latitude_x,Longitude_x,point_x,"NOx, мг/м³_x","Фенол, мг/м³_x","Бензол, мг/м³_x","Толуол, мг/м³_x","Этилбензол, мг/м³_x","М,п-ксилол, мг/м³_x","О-ксилол, мг/м³_x","Стирол, мг/м³_x","Хлорбензол, мг/м³_x","Этилена оксид, мг/м³_x","CH4, мг/м³_x","Этан, мг/м³_x","Этилен, мг/м³_x","Пропен, мг/м³_x","Пропан, мг/м³_x","Изобутан, мг/м³_x","Бутан, мг/м³_x","Изобутилен, мг/м³_x","Бутен-1, мг/м³_x","Бутен-2, мг/м³_x","Пентан, мг/м³_x","Изопентан, мг/м³_x","Гексан, мг/м³_x","Гептан, мг/м³_x","Октан, мг/м³_x","Нонан, мг/м³_x","Декан, мг/м³_x","О,м,п-ксилолы, мг/м³_x","М-ксилол, мг/м³_x","П-ксилол, мг/м³_x","Формальдегид, мг/м³_x","CO2, мг/м³_x","Нафталин, мг/м³_x","Бутилацетат, мг/м³_x","Бутанол, мг/м³_x",ind,Unnamed: 0_y,Показатель:_y,"CO, мг/м³_y","NO, мг/м³_y","NO2, мг/м³_y","NH3, мг/м³_y","SO2, мг/м³_y","H2S, мг/м³_y","T внеш., °C_y","V ветра, м/с_y","D ветра, °_y","H внеш., %_y","P атм., мм.рт.ст._y","R осадк., мм/ч_y",Latitude_y,Longitude_y,point_y,"NOx, мг/м³_y","Фенол, мг/м³_y","Бензол, мг/м³_y","Толуол, мг/м³_y","Этилбензол, мг/м³_y","М,п-ксилол, мг/м³_y","О-ксилол, мг/м³_y","Стирол, мг/м³_y","Хлорбензол, мг/м³_y","Этилена оксид, мг/м³_y","CH4, мг/м³_y","Этан, мг/м³_y","Этилен, мг/м³_y","Пропен, мг/м³_y","Пропан, мг/м³_y","Изобутан, мг/м³_y","Бутан, мг/м³_y","Изобутилен, мг/м³_y","Бутен-1, мг/м³_y","Бутен-2, мг/м³_y","Пентан, мг/м³_y","Изопентан, мг/м³_y","Гексан, мг/м³_y","Гептан, мг/м³_y","Октан, мг/м³_y","Нонан, мг/м³_y","Декан, мг/м³_y","О,м,п-ксилолы, мг/м³_y","М-ксилол, мг/м³_y","П-ксилол, мг/м³_y","Формальдегид, мг/м³_y","CO2, мг/м³_y","Нафталин, мг/м³_y","Бутилацетат, мг/м³_y","Бутанол, мг/м³_y"
115,13.06.2023 22:30,NaN,0.301,NaN,NaN,NaN,"0,0094","0,0048","10,9","0,0",65.0,58,"758,2","0,0",55.598983,51.771936,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",NaN,NaN,NaN,NaN,115,13.06.2023 22:30,NaN,1.3,"0,028","0,033","0,000","0,0053","0,0026","11,3","0,4",65.0,52,"756,8","0,0",55.622944,51.82557,F,NaN,"0,000","0,000","0,000","0,000","0,000","0,000","0,000","0,000","0,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",NaN,"0,000","0,000","0,000"
116,13.06.2023 22:30,NaN,0.301,NaN,NaN,NaN,"0,0094","0,0048","10,9","0,0",65.0,58,"758,2","0,0",55.598983,51.771936,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",NaN,NaN,NaN,NaN,116,13.06.2023 23:00,NaN,1.2,"0,028","0,034","0,000","0,0057","0,0025","10,2","0,1",63.0,55,"756,7","0,0",55.622944,51.82557,F,NaN,"0,000","0,000","0,000","0,000","0,000","0,000","0,000","0,000","0,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",NaN,"0,000","0,000","0,000"
117,13.06.2023 22:30,NaN,0.301,NaN,NaN,NaN,"0,0094","0,0048","10,9","0,0",65.0,58,"758,2","0,0",55.598983,51.771936,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",NaN,NaN,NaN,NaN,117,13.06.2023 23:30,NaN,0.6,"0,026","0,034","0,000","0,0056","0,0026","9,7","0,0",NaN,57,"756,6","0,0",55.622944,51.82557,F,NaN,"0,000","0,000","0,000","0,000","0,000","0,000","0,000","0,000","0,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",NaN,"0,000","0,000","0,000"


In [288]:
# result['NO, мг/м³'] = result['NO, мг/м³'].str.replace(',', '.')
# result['NO, мг/м³'] = pd.to_numeric(result['NO, мг/м³'], errors='coerce')
# hdf = result[result['NO, мг/м³'] > result['NO, мг/м³'].quantile(0.8)]

In [289]:
# hdf.groupby('point').count()

,Unnamed: 0,Показатель:,"CO, мг/м³","NO, мг/м³","NO2, мг/м³","NH3, мг/м³","SO2, мг/м³","H2S, мг/м³","T внеш., °C","V ветра, м/с","D ветра, °","H внеш., %","P атм., мм.рт.ст.","R осадк., мм/ч",Latitude,Longitude,"NOx, мг/м³","Фенол, мг/м³","Бензол, мг/м³","Толуол, мг/м³","Этилбензол, мг/м³","М,п-ксилол, мг/м³","О-ксилол, мг/м³","Стирол, мг/м³","Хлорбензол, мг/м³","Этилена оксид, мг/м³","CH4, мг/м³","Этан, мг/м³","Этилен, мг/м³","Пропен, мг/м³","Пропан, мг/м³","Изобутан, мг/м³","Бутан, мг/м³","Изобутилен, мг/м³","Бутен-1, мг/м³","Бутен-2, мг/м³","Пентан, мг/м³","Изопентан, мг/м³","Гексан, мг/м³","Гептан, мг/м³","Октан, мг/м³","Нонан, мг/м³","Декан, мг/м³","О,м,п-ксилолы, мг/м³","М-ксилол, мг/м³","П-ксилол, мг/м³","Формальдегид, мг/м³","CO2, мг/м³","Нафталин, мг/м³","Бутилацетат, мг/м³","Бутанол, мг/м³"
point,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A,4,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
B,4,0,4,4,4,4,2,2,4,4,4,4,4,4,4,4,4,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,66,0,9,66,66,66,65,65,66,66,66,66,66,66,66,66,66,18,18,18,18,0,18,18,18,49,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,18,18,18,0,0,0,0,0
D,103,0,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103,0,0,0,0
E,293,0,101,293,293,293,0,0,293,293,293,293,293,293,293,293,293,260,260,260,260,260,260,260,260,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,293,293,260,260,260
F,734,0,734,734,734,734,734,734,734,734,734,734,734,734,734,734,0,727,553,553,553,553,553,553,553,732,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,688,0,727,553,553


In [291]:
# result.head()
# result['Unnamed: 0'] = pd.to_datetime(result['Unnamed: 0'])

In [293]:
# sorted_df = result.sort_values(by='Unnamed: 0', ascending=False)

In [294]:
# sorted_df

,Unnamed: 0,Показатель:,"CO, мг/м³","NO, мг/м³","NO2, мг/м³","NH3, мг/м³","SO2, мг/м³","H2S, мг/м³","T внеш., °C","V ветра, м/с","D ветра, °","H внеш., %","P атм., мм.рт.ст.","R осадк., мм/ч",Latitude,Longitude,point,"NOx, мг/м³","Фенол, мг/м³","Бензол, мг/м³","Толуол, мг/м³","Этилбензол, мг/м³","М,п-ксилол, мг/м³","О-ксилол, мг/м³","Стирол, мг/м³","Хлорбензол, мг/м³","Этилена оксид, мг/м³","CH4, мг/м³","Этан, мг/м³","Этилен, мг/м³","Пропен, мг/м³","Пропан, мг/м³","Изобутан, мг/м³","Бутан, мг/м³","Изобутилен, мг/м³","Бутен-1, мг/м³","Бутен-2, мг/м³","Пентан, мг/м³","Изопентан, мг/м³","Гексан, мг/м³","Гептан, мг/м³","Октан, мг/м³","Нонан, мг/м³","Декан, мг/м³","О,м,п-ксилолы, мг/м³","М-ксилол, мг/м³","П-ксилол, мг/м³","Формальдегид, мг/м³","CO2, мг/м³","Нафталин, мг/м³","Бутилацетат, мг/м³","Бутанол, мг/м³"
4751,2023-12-06 23:30:00,NaN,0.249,NaN,NaN,NaN,"0,0085","0,0037","11,2","0,5",85 (В),56,"761,2","0,0",55.598983,51.771936,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",NaN,NaN,NaN,NaN
7535,2023-12-06 23:30:00,NaN,0.300,0.027,"0,026","0,000","0,0052","0,0020","6,8","0,3",72 (В),65,"759,8","0,0",55.622944,51.825570,F,NaN,"0,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",NaN,"0,000",NaN,NaN
575,2023-12-06 23:30:00,NaN,0.900,0.000,"0,029","0,015","0,023","0,0031","12,7","2,4",50 (С-В),58,"759,0","0,0",55.539306,51.856451,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6143,2023-12-06 23:30:00,NaN,NaN,0.010,"0,055","0,000",NaN,NaN,"11,3","0,4",330 (С-З),50,"761,6","0,0",55.650091,51.852687,E,"0,065","0,000","0,000","0,000","0,000","0,000","0,000","0,000","0,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",16,"0,000","0,000","0,000"
3359,2023-12-06 23:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"11,9","1,8",113 (Ю-В),50,"760,3","0,0",55.613193,51.784821,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6960,2023-01-06 00:00:00,NaN,0.100,0.002,"0,032","0,006","0,0064","0,0026","18,7","0,7",341 (С),66,"745,7","0,0",55.622944,51.825570,F,NaN,"0,000","0,000","0,000","0,000","0,000","0,000","0,000","0,000","0,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",NaN,"0,000","0,000","0,000"
5568,2023-01-06 00:00:00,NaN,0.100,0.014,"0,043","0,003",NaN,NaN,"19,2","1,3",212 (Ю-З),64,"747,4","0,0",55.650091,51.852687,E,"0,057","0,000","0,000","0,000","0,000","0,000","0,000","0,000","0,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000",1,"0,000","0,000","0,000"
2784,2023-01-06 00:00:00,NaN,0.357,NaN,NaN,NaN,"0,0138","0,0053","17,5","0,5",198 (Ю),76,"746,1","0,0",55.613193,51.784821,C,NaN,"0,00017","0,00000","0,00321","0,00000",NaN,"0,00000","0,00003","0,00002",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,00187","0,00095","0,00092",NaN,NaN,NaN,NaN,NaN
1392,2023-01-06 00:00:00,NaN,0.600,0.000,"0,010","0,001","0,0027","0,0023","19,9","0,7",177 (Ю),64,"745,6","0,0",55.654578,51.800072,B,"0,010",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,093",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
